In [ ]:
%load_ext autoreload
%autoreload 2

import os
import uproot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats
import datetime as dt
import scipy.optimize

import importlib

from datetime import datetime

# local imports
from lib.constants import *

In [ ]:
dosave = True
savedir = "./plots/"
plt.rcParams.update({'font.size': 14})
plotqual = "_run%i"
plottitle = "Run %i"
tpcnames = ["EE", "EW", "WE", "WW"]

datadir = "/icarus/data/users/gputnam/MCP2022G/data/drift-normalization/"

save_datadir = "./data/"
savedata = False
outf = save_datadir + "drift_times_witherr.txt"

In [ ]:
rundate = [
"7139, 11/18/2021 6:33:05",
"7152, 11/18/2021 17:27:56",
"7178, 11/20/2021 15:20:35",
"7190, 11/22/2021 15:40:33",
"7209, 11/24/2021 10:03:18",
"7230, 11/28/2021 8:16:34",
"7244, 12/4/2021 11:43:09",
"7262, 12/7/2021 17:04:30",
"7334, 12/10/2021 20:47:24",
"7339, 12/11/2021 17:00:57",
"7418, 12/26/2021 1:45:12",
"7428, 12/31/2021 14:28:10",
"7471, 1/16/2022 9:14:35",
"7504, 1/24/2022 13:24:00",
"7564, 1/30/2022 3:49:04",
"7587, 2/5/2022 15:48:04",
"7609, 2/11/2022 20:37:09",
"7615, 2/14/2022 21:07:23",
"7729, 2/23/2022 21:36:17",
"7755, 2/24/2022 19:51:26",
"7819, 2/28/2022 20:30:09",
"7840, 3/2/2022 18:21:24",
"7844, 3/4/2022 12:35:27",
"7897, 3/13/2022 9:56:26",
]

run2date = {}

for rd in rundate:
    run, date = tuple(rd.split(","))
    run = int(run)
    date = datetime.strptime(date.split(" ")[1], "%m/%d/%Y")
    
    run2date[run] = date


In [ ]:
[f for f in os.listdir(datadir) if f.endswith(".df") and f.startswith("driftV")]

In [ ]:
dfs = [pd.read_hdf(datadir + f) for f in os.listdir(datadir) if f.endswith(".df") and f.startswith("driftV")]

In [ ]:
data = pd.concat(dfs)

In [ ]:
data

In [ ]:
del dfs

In [ ]:
runs = np.array(sorted(data.meta.run.unique()))
cryostats = data.cryostat.unique()
cryostat_names = ["E", "W"]
tpcnames = ["EE", "EW", "WE", "WW"]

In [ ]:
min(runs), max(runs)

In [ ]:
deltat_E = data.hit_max_time_p2_tpcE - data.hit_min_time_p2_tpcE
deltat_W = data.hit_max_time_p2_tpcW - data.hit_min_time_p2_tpcW

tstep_E = ((data.tpcE_max.dt + data.tpcE_min.dt))
tstep_W = ((data.tpcW_max.dt + data.tpcW_min.dt))

In [ ]:
# define the fit function
def fit_readout_window(tick, *p):
    N = p[0]
    sigma = p[1]
    center = p[2]
    beta = p[3]
    m = p[4]
    return N*stats.crystalball.pdf(tick, beta, m, center, sigma)

In [ ]:
bins = np.linspace(2200, 2500, 61)*tick_period/1e3
bin_center = (bins[1:] + bins[:-1]) / 2.
points = np.linspace(bins[0], bins[-1], 101)

In [ ]:
run_is_valid = np.array([all([
        (np.histogram(deltat_E[(data.meta.run == r) & (data.cryostat == c)]*tick_period/1e3, bins=bins)[0].max() > 100) &\
        (np.histogram(deltat_W[(data.meta.run == r) & (data.cryostat == c)]*tick_period/1e3, bins=bins)[0].max() > 100)
                    for c in cryostats])
               for r in runs])

In [ ]:
drift_time = {}
drift_time_err = {}
N_vel = {}

for tpc in tpcnames:
    drift_time[tpc] = []
    drift_time_err[tpc] = []
    N_vel[tpc] = []

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
colors = ["#CC3351", "#33CCAE"]

In [ ]:
valid_runs = np.array(runs)[run_is_valid]

In [ ]:
mpl.rcParams['mathtext.default'] = 'regular'

ifig = 0

for c in cryostats:
    cname = cryostat_names[c]
    for r in valid_runs:
        plt.figure(ifig)
        ifig += 1
        
        when = (data.meta.run == r) & (data.cryostat == c)
        NE,_,_ = plt.hist(deltat_E[when]*tick_period/1e3, bins=bins, histtype="step", label="TPC E Data", linewidth=2, color=colors[0])
        NW,_,_ = plt.hist(deltat_W[when]*tick_period/1e3, bins=bins, histtype="step", label="TPC W Data", linewidth=2, color=colors[1])

        p0 = [3000, 10, 2350.*tick_period/1e3, 1, 2.]
        popt, perr = curve_fit(fit_readout_window, bin_center[NE>0], NE[NE>0], sigma=np.sqrt(NE[NE>0]), p0=p0, maxfev=int(1e5))
            
        plt.plot(points, fit_readout_window(points, *popt), label="TPC E Fit", 
                 color="black", linewidth=1.5, linestyle="--")
        drift_length_E = popt[2]
        drift_length_E_err = np.sqrt(np.diag(perr))[2]
        
        p0 = [5e5, 10, 2350.*tick_period/1e3, 1, 2.]
        popt, perr = curve_fit(fit_readout_window, bin_center[NW>0], NW[NW>0], sigma=np.sqrt(NW[NW>0]), p0=p0, maxfev=int(1e5))
            
        plt.plot(points, fit_readout_window(points, *popt), label="TPC W Fit", 
                 color="black", linewidth=1.5, linestyle=":")
        drift_length_W = popt[2]
        drift_length_W_err = np.sqrt(np.diag(perr))[2]
        
        drift_time_E = (drift_length_E )
        drift_time_W = (drift_length_W )
        
        drift_time_E_err = drift_length_E_err
        drift_time_W_err = drift_length_W_err

        handles, labels = plt.gca().get_legend_handles_labels()
        order = [2,0,3,1]
        plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], loc="upper left")
        
        plt.text(0.65, 0.3, "Run %i\nTPC E Drift Time:\n%.1f $\\mu$s\n\nTPC W Drift Time:\n%.1f $\\mu$s" % (r, drift_time_E, drift_time_W), transform=plt.gca().transAxes, fontsize=14)

        plt.ylabel("Tracks")
        plt.xlabel("Maximum Ionization Drift Time [$\mu$s]")
        # plt.title("Run %i Cryostat %s" % (r, cname))
        plt.title("Ionization Drift Time: %s Cryostat" % ("West" if cname == "W" else "East"))
        plt.tight_layout()
        
        if dosave: plt.savefig(savedir + "driftV_run%i_cryo%s.pdf" % (r, cname))
        if dosave: plt.savefig(savedir + "driftV_run%i_cryo%s.png" % (r, cname))
        
        drift_time[cname + "E"].append(drift_time_E)
        drift_time[cname + "W"].append(drift_time_W)
        
        drift_time_err[cname + "E"].append(drift_time_E_err)
        drift_time_err[cname + "W"].append(drift_time_W_err)
        
        N_vel[cname + "E"].append(np.sum(deltat_E[when] > bins[0]))
        N_vel[cname + "W"].append(np.sum(deltat_W[when] > bins[0]))
        
        


In [ ]:
drift_vel = {}
drift_vel_err = {}

for tpc in tpcnames:
    drift_time[tpc] = np.array(drift_time[tpc])
    drift_time_err[tpc] = np.array(drift_time_err[tpc])
    N_vel[tpc] = np.array(N_vel[tpc])
    
    drift_vel[tpc] = a2c_dist / drift_time[tpc]
    drift_vel_err[tpc] = drift_vel[tpc] * (drift_time_err[tpc] / drift_time[tpc])

In [ ]:
plt.errorbar(valid_runs, drift_vel["EE"], drift_vel_err["EE"], label="EE")
plt.errorbar(valid_runs, drift_vel["EW"], drift_vel_err["EW"], label="EW")
plt.xlabel("Run Number")
plt.ylabel("Drift Velocity [cm/$\\mu$s]")
plt.legend()
plt.tight_layout()

if dosave: plt.savefig(savedir + "drift_velocity_run_cryoE.pdf")

In [ ]:
datedruns = [r in run2date for r in valid_runs]

In [ ]:
plt.errorbar([run2date[r] for r in valid_runs[datedruns]], drift_vel["EE"][datedruns], 
             drift_vel_err["EE"][datedruns], label="EE")
plt.errorbar([run2date[r] for r in valid_runs[datedruns]], drift_vel["EW"][datedruns], 
             drift_vel_err["EW"][datedruns], label="EW")
plt.xlabel("Run Date")
plt.ylabel("Drift Velocity [cm/$\\mu$s]")
plt.xticks(rotation=30, ha='right')
plt.legend()
plt.tight_layout()

if dosave: plt.savefig(savedir + "drift_velocity_rundate_cryoE.pdf")

In [ ]:
plt.errorbar(valid_runs, drift_vel["WE"], drift_vel_err["WE"], label="WE")
plt.errorbar(valid_runs, drift_vel["WW"], drift_vel_err["WW"], label="WW")
plt.xlabel("Run Number")
plt.ylabel("Drift Velocity [cm/$\\mu$s]")
plt.legend()
plt.tight_layout()

if dosave: plt.savefig(savedir + "drift_velocity_run_cryoW.pdf")

In [ ]:
plt.errorbar([run2date[r] for r in valid_runs[datedruns]], drift_vel["WE"][datedruns], 
             drift_vel_err["WE"][datedruns], label="WE")
plt.errorbar([run2date[r] for r in valid_runs[datedruns]], drift_vel["WW"][datedruns], 
             drift_vel_err["WW"][datedruns], label="WW")
plt.xlabel("Run Date")
plt.ylabel("Drift Velocity [cm/$\\mu$s]")
plt.xticks(rotation=30, ha='right')
plt.legend()
plt.tight_layout()

if dosave: plt.savefig(savedir + "drift_velocity_rundate_cryoW.pdf")

In [ ]:
def mean_w_err(means, errs):
    means = np.array(means)
    errs = np.array(errs)
    mean = np.sum(means/errs**2, axis=0) / np.sum(1/errs**2, axis=0)
    err = np.sqrt(1/ np.sum(1/errs**2, axis=0))
    return mean, err

In [ ]:
if savedata:
    with open(outf.replace(".txt", "_nocorr.txt"), "w") as f:
        f.write("Drift Length [us]\n")
        f.write("Run" + "\t" + "\t".join(tpcnames) + "\tStat-Err-" + "\tStat-Err-".join(tpcnames) + "\tSyst-Err" + "\n")
        for d in zip(valid_runs, *([drift_time[t] for t in tpcnames]), *([drift_time_err[t] for t in tpcnames]),):
            f.write("\t".join(map(str, d)) + "\t" + str(0) + "\n")
            

In [ ]:
plt.figure(figsize=(6, 9))
when = (data.cryostat == 0)
vary = data.tpcE_max.z[when]
varx = data.tpcE_max.time[when] - data.tpcE_min.time[when]

timelo = 2320 
timehi = 2390

biny = np.linspace(-900, 900, 101)
binx = np.linspace(timelo, timehi, 21)

N,_,_,_ = plt.hist2d(varx, vary, bins=[binx, biny])

Z = (biny[1:] + biny[:-1]) / 2.
X = ((binx[1:] + binx[:-1]) / 2.)[np.argmax(N,axis=0)]
#plt.plot(X, Z, color="red")#, linestyle=":")

def tick2cm(t):
    return (t - (timehi+timelo)/2.) * tick_period*1e-3 * np.mean(drift_vel["EE"])
def cm2tick(x):
    return (x / (tick_period*1e-3*np.mean(drift_vel["EE"]))) + (timehi+timelo)/2.

plt.xlabel("Track Time Length [ticks]")

axtop = plt.gca().secondary_xaxis("top", functions=(tick2cm, cm2tick))

c = plt.colorbar()
plt.ylabel("Max Time Hit Detector Z [cm]")
axtop.set_xlabel("Track Time Length [cm]")
plt.title("TPC EE All Runs")
plt.tight_layout()

if dosave: plt.savefig(savedir + "cathode_bending_tpcEE.pdf")

In [ ]:
plt.figure(figsize=(6, 9))

when = (data.cryostat == 0)
vary = data.tpcW_max.z[when]
varx = data.tpcW_max.time[when] - data.tpcW_min.time[when]

timelo = 2320 
timehi = 2390

biny = np.linspace(-900, 900, 101)
binx = np.linspace(timelo, timehi, 21)

N,_,_,_ = plt.hist2d(varx, vary, bins=[binx, biny])

Z = (biny[1:] + biny[:-1]) / 2.
X = ((binx[1:] + binx[:-1]) / 2.)[np.argmax(N,axis=0)]
#plt.plot(X, Z, color="red")#, linestyle=":")

def tick2cm(t):
    return (t - (timehi+timelo)/2.) * tick_period*1e-3 * np.mean(drift_vel["EW"])
def cm2tick(x):
    return (x / (tick_period*1e-3*np.mean(drift_vel["EW"]))) + (timehi+timelo)/2.

plt.xlabel("Track Time Length [ticks]")

axtop = plt.gca().secondary_xaxis("top", functions=(tick2cm, cm2tick))

c = plt.colorbar()
plt.ylabel("Max Time Hit Detector Z [cm]")
axtop.set_xlabel("Track Time Length [cm]")
plt.title("TPC EW All Runs")
plt.tight_layout()

if dosave: plt.savefig(savedir + "cathode_bending_tpcEW.pdf")

In [ ]:
plt.figure(figsize=(6, 9))

when = (data.cryostat == 1)
vary = data.tpcE_max.z[when]
varx = data.tpcE_max.time[when] - data.tpcE_min.time[when]

timelo = 2320 
timehi = 2390

biny = np.linspace(-900, 900, 101)
binx = np.linspace(timelo, timehi, 21)

N,_,_,_ = plt.hist2d(varx, vary, bins=[binx, biny])

Z = (biny[1:] + biny[:-1]) / 2.
X = ((binx[1:] + binx[:-1]) / 2.)[np.argmax(N,axis=0)]
#plt.plot(X, Z, color="red")#, linestyle=":")

def tick2cm(t):
    return (t - (timehi+timelo)/2.) * tick_period*1e-3 * np.mean(drift_vel["WE"])
def cm2tick(x):
    return (x / (tick_period*1e-3*np.mean(drift_vel["WE"]))) + (timehi+timelo)/2.

plt.xlabel("Track Time Length [ticks]")

axtop = plt.gca().secondary_xaxis("top", functions=(tick2cm, cm2tick))

c = plt.colorbar()
plt.ylabel("Max Time Hit Detector Z [cm]")
axtop.set_xlabel("Track Time Length [cm]")
plt.title("TPC WE All Runs")
plt.tight_layout()

if dosave: plt.savefig(savedir + "cathode_bending_tpcWE.pdf")

In [ ]:
plt.figure(figsize=(6, 9))

when = (data.cryostat == 1)
vary = data.tpcW_max.z[when]
varx = data.tpcW_max.time[when] - data.tpcW_min.time[when]

timelo = 2320 
timehi = 2390

biny = np.linspace(-900, 900, 101)
binx = np.linspace(timelo, timehi, 21)

N,_,_,_ = plt.hist2d(varx, vary, bins=[binx, biny])

Z = (biny[1:] + biny[:-1]) / 2.
X = ((binx[1:] + binx[:-1]) / 2.)[np.argmax(N,axis=0)]
#plt.plot(X, Z, color="red")#, linestyle=":")

def tick2cm(t):
    return (t - (timehi+timelo)/2.) * tick_period*1e-3 * np.mean(drift_vel["WW"])
def cm2tick(x):
    return (x / (tick_period*1e-3*np.mean(drift_vel["WW"]))) + (timehi+timelo)/2.

plt.xlabel("Track Time Length [ticks]")

axtop = plt.gca().secondary_xaxis("top", functions=(tick2cm, cm2tick))

c = plt.colorbar()
plt.ylabel("Max Time Hit Detector Z [cm]")
axtop.set_xlabel("Track Time Length [cm]")
plt.title("TPC WW All Runs")
plt.tight_layout()

if dosave: plt.savefig(savedir + "cathode_bending_tpcWW.pdf")

In [ ]:
def lin(x, m, b):
    return m*x+b

In [ ]:
slopes_tpcE = []
slopes_tpcW = []

slopes_err_tpcE = []
slopes_err_tpcW = []

In [ ]:
biny = np.linspace(2200, 2500, 61)*tick_period/1e3
biny_center = (biny[1:] + biny[:-1]) / 2.
binx = np.linspace(0, 20, 11)*tick_period/1e3
binx_center = (binx[1:] + binx[:-1]) / 2.

points = np.linspace(bins[0], bins[-1], 101)

ifig = 0

for c in cryostats:
    cname = cryostat_names[c]
    plt.figure(ifig)
    when = (data.cryostat == c)
    NE,_, _,_ = plt.hist2d(tstep_E[when]*tick_period/1e3, deltat_E[when]*tick_period/1e3, bins=[binx, biny])
    
    peaks = []
    peak_errs = []
    FITBIN0 = 2
    for i in range(FITBIN0, len(binx)-1):
        p0 = [3000, 10, 2350.*tick_period/1e3, 1, 2.]
        popt, perr = curve_fit(fit_readout_window, biny_center[NE[i]>0], NE[i][NE[i]>0], sigma=np.sqrt(NE[i][NE[i]>0]), p0=p0, maxfev=int(1e5))
        peaks.append(popt[2])
        peak_errs.append(np.sqrt(np.diag(perr))[2])    
        
    plt.errorbar(binx_center[FITBIN0:], peaks, peak_errs, color="r")
    popt,perr = curve_fit(lin, binx_center[FITBIN0:], peaks, sigma=peak_errs, p0=[950, -0.1])
    slope = popt[0]
    plt.text(0.05, 0.05, "Slope: %.3f" % slope, color="w", transform=plt.gca().transAxes)
    slopes_tpcE.append(slope)
    slopes_err_tpcE.append(np.sqrt(np.diag(perr))[0])
    
    plt.ylim([920, 960])
    plt.title("TPC %sE" % (cname))
    plt.ylabel("Hit Time Length [ticks]")
    plt.xlabel("Averge Time Between Hits [ticks]")
    plt.colorbar()
    plt.tight_layout()

    if dosave: plt.savefig(savedir + "TPC%s_dt_timelength.pdf" % (cname + "E",))
    ifig += 1

    plt.figure(ifig)     
    plt.title("TPC %sW" % (cname))
    NW,_,_,_ = plt.hist2d(tstep_W[when]*tick_period/1e3, deltat_W[when]*tick_period/1e3, bins=[binx, biny])
    
    peaks = []
    peak_errs = []
    FITBIN0 = 2
    for i in range(FITBIN0, len(binx)-1):
        p0 = [3000, 10, 2350.*tick_period/1e3, 1, 2.]
        popt, perr = curve_fit(fit_readout_window, biny_center[NW[i]>0], NW[i][NW[i]>0], sigma=np.sqrt(NW[i][NW[i]>0]), p0=p0, maxfev=int(1e5))
        peaks.append(popt[2])
        peak_errs.append(np.sqrt(np.diag(perr))[2])    
        
    plt.errorbar(binx_center[FITBIN0:], peaks, peak_errs, color="r")
    popt,perr = curve_fit(lin, binx_center[FITBIN0:], peaks, sigma=peak_errs, p0=[950, -0.1])
    slope = popt[0]
    plt.text(0.05, 0.05, "Slope: %.3f" % slope, color="w", transform=plt.gca().transAxes)
    slopes_tpcW.append(slope)
    slopes_err_tpcW.append(np.sqrt(np.diag(perr))[0])

    
    plt.ylim([920, 960])
    ifig += 1
    plt.ylabel("Hit Time Length [$\\mu$s]")
    plt.xlabel("Averge Time Between Hits [$\\mu$s]")
    plt.colorbar()
    plt.tight_layout()

    if dosave: plt.savefig(savedir + "TPC%s_dt_timelength.pdf" % (cname + "W",))


In [ ]:
deltat_E_corr = deltat_E - tstep_E*slopes_tpcE[0]
deltat_E_corr[data.cryostat == 1] = (deltat_E - tstep_E*slopes_tpcE[1])[data.cryostat == 1]

deltat_W_corr = deltat_W - tstep_W*slopes_tpcW[0]
deltat_W_corr[data.cryostat == 1] = (deltat_W - tstep_W*slopes_tpcW[1])[data.cryostat == 1]

In [ ]:
drift_time_corr = {}
drift_time_corr_err = {}

for tpc in tpcnames:
    drift_time_corr[tpc] = []
    drift_time_corr_err[tpc] = []

In [ ]:
mpl.rcParams['mathtext.default'] = 'regular'

ifig = 0

for c in cryostats:
    cname = cryostat_names[c]
    for r in valid_runs:
        plt.figure(ifig)
        ifig += 1
        
        when = (data.meta.run == r) & (data.cryostat == c)
        NE,_,_ = plt.hist(deltat_E_corr[when]*tick_period/1e3, bins=bins, histtype="step", label="TPC E Data", linewidth=2, color=colors[0])
        NW,_,_ = plt.hist(deltat_W_corr[when]*tick_period/1e3, bins=bins, histtype="step", label="TPC W Data", linewidth=2, color=colors[1])

        p0 = [3000, 10, 2350.*tick_period/1e3, 1, 2.]
        popt, perr = curve_fit(fit_readout_window, bin_center[NE>0], NE[NE>0], sigma=np.sqrt(NE[NE>0]), p0=p0, maxfev=int(1e5))
            
        plt.plot(points, fit_readout_window(points, *popt), label="TPC E Fit", 
                 color="black", linewidth=1.5, linestyle="--")
        drift_length_E = popt[2]
        drift_length_E_err = np.sqrt(np.diag(perr))[2]
        
        p0 = [5e5, 10, 2350.*tick_period/1e3, 1, 2.]
        popt, perr = curve_fit(fit_readout_window, bin_center[NW>0], NW[NW>0], sigma=np.sqrt(NW[NW>0]), p0=p0, maxfev=int(1e5))
            
        plt.plot(points, fit_readout_window(points, *popt), label="TPC W Fit", 
                 color="black", linewidth=1.5, linestyle=":")
        drift_length_W = popt[2]
        drift_length_W_err = np.sqrt(np.diag(perr))[2]

        handles, labels = plt.gca().get_legend_handles_labels()
        order = [2,0,3,1]
        plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], loc="upper left")
        
        plt.text(0.65, 0.3, "Run %i\nTPC E Drift Time:\n%.1f $\\mu$s\n\nTPC W Drift Time:\n%.1f $\\mu$s" % (r, drift_time_E, drift_time_W), transform=plt.gca().transAxes, fontsize=14)

        plt.ylabel("Tracks")
        plt.xlabel("Maximum Ionization Drift Time [$\mu$s]")
        # plt.title("Run %i Cryostat %s" % (r, cname))
        plt.title("Ionization Drift Time: %s Cryostat" % ("West" if cname == "W" else "East"))
        plt.tight_layout()
        
        if dosave: plt.savefig(savedir + "driftV_corr_run%i_cryo%s.pdf" % (r, cname))
        if dosave: plt.savefig(savedir + "driftV_corr_run%i_cryo%s.png" % (r, cname))
        
        drift_time_corr[cname + "E"].append(drift_length_E)
        drift_time_corr[cname + "W"].append(drift_length_W)
        
        drift_time_corr_err[cname + "E"].append(drift_length_E_err)
        drift_time_corr_err[cname + "W"].append(drift_length_W_err)
        

In [ ]:
if savedata:
    with open(outf.replace(".txt", "_corr.txt"), "w") as f:
        f.write("Drift Length [us]\n")
        f.write("Run" + "\t" + "\t".join(tpcnames) + "\tStat-Err-" + "\tStat-Err-".join(tpcnames) + "\tSyst-Err" + "\n")
        for d in zip(valid_runs, *([drift_length_corr[t] for t in tpcnames]), *([drift_length_corr_err[t] for t in tpcnames]),):
            f.write("\t".join(map(str, d)) + "\t" + str(0.5*tick_period/1e3) + "\n")
            